# Parallel Corpora for Seq2seq Generation Training

In [2]:
import pandas as pd
import numpy as np

from tqdm.auto import tqdm
import astunparse

In [3]:
sample_df = pd.read_csv("/data/users/cting3/CodeNest/code-style-probing/data/eval_parallel_corpora/eval_set_individual_feat.csv")

In [6]:
df = sample_df

In [10]:
comment_df = df[["content", "uncommented_content"]].copy().rename(columns={"content": "X", "uncommented_content": "Y"})
class_df = df[["uncommented_content", "no_class_content"]].copy().rename(columns={"uncommented_content": "X", "no_class_content": "Y"})
docstring_df = df[["uncommented_content", "no_docstring_content"]].copy().rename(columns={"uncommented_content": "X", "no_docstring_content": "Y"})
list_comp_df = df[["uncommented_content", "no_comp_content"]].copy().rename(columns={"uncommented_content": "X", "no_comp_content": "Y"})
casing_df = df[["uncommented_content", "no_casing_content"]].copy().rename(columns={"uncommented_content": "X", "no_casing_content": "Y"})

df_list = [
    comment_df, class_df,docstring_df, list_comp_df, casing_df
]



In [15]:
for idx, feature_df in enumerate(df_list):
    feature_df = feature_df[feature_df["X"].notnull()]
    feature_df = feature_df[feature_df["Y"].notnull()]
    feature_df = feature_df.query(f"X != Y")
    print(len(feature_df))
    df_list[idx] = feature_df

279611
156637
75999
21935
235040


In [16]:
[len(feat_df) for feat_df in df_list]

[279611, 156637, 75999, 21935, 235040]

In [7]:
uncommented_df = df[["content", "uncommented_content"]].rename(columns={"content": "X", "uncommented_content": "Y"})

In [8]:
uncommented_df

,X,Y
0,import jinja2\nfrom gofedlib.utils import getS...,NaN
1,"from collections import Counter\nwith open(""/h...",NaN
2,__author__ = 'tonycastronova'\n\nimport dateti...,NaN
3,"#!/usr/bin/env python3\n# -*- coding: utf-8, v...",\n__author__ = 'morta@digitus.itk.ppke.hu'\nim...
4,# bo.zhang@ki.se\n# Chimeric MS/MS estimation ...,NaN
...,...,...
393643,# Geopy will get the distance between pairs of...,\nimport geopy\nbase_path = '.'\nfrom pyspark....
393644,f = CurrentFont()\ng = CurrentGlyph()\n\nlayer...,\nf = CurrentFont()\ng = CurrentGlyph()\nlayer...
393645,# -*- coding: utf-8 -*-\n# Copyright 2016 Yelp...,\nfrom __future__ import absolute_import\nimpo...
393646,#!/usr/bin/python\n\n# Python library for Adaf...,NaN


In [ ]:
import numpy as np
import pandas as pd
from transformers import *
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from datasets import Dataset
import pickle
from utils.helper import read_py150k_code, read_file_to_string
import regex as re

fname_prefix = "/data/users/cting3/CodeNest/code-style-probing/"
feat_name = "no_class_no_super"

# csv_fname = "eval_set_individual_feat.csv"


# df = pd.read_csv(fname_prefix + f"data/eval_parallel_corpora/{csv_fname}")

uncommented_df = df[["content", "uncommented_content"]]

In [2]:
eval_set_df = pd.read_csv("data/evaluation_set.csv")

## Comments

In [2]:
fname_prefix = "/data/users/team2_capstone/code-style-probing/"
test = "class MyClass: \n\t\"\"\"A simple example class\"\"\" \n\ti = 12345 # le epic comment\n\tWhoa = [i + 1 for i in range(0,10)]\n\n\tdef f(self):\n\t\treturn 'hello world'"
print(test)

class MyClass: 
	"""A simple example class""" 
	i = 12345 # le epic comment
	whoa = [i + 1 for i in range(0,10)]

	def f(self):
		return 'hello world'


In [3]:
import ast
def uncomment(source):
    """ 
    Takes input code and returns code with comments stripped
    Input: code (str)
    Output: code (str)
    """
    try:
        parse = ast.unparse(ast.parse(source))
    except:
        parse = 'nan'
    return parse

In [4]:
uncomment(test)

NameError: name 'test' is not defined

In [1]:
import pandas as pd
bq_df = pd.read_csv("data/labeled_code/combined_data_uncommented.csv")

/tmp/ipykernel_204716/247637156.py:2: DtypeWarning: Columns (84,85,86,87) have mixed types. Specify dtype option on import or set low_memory=False.
  bq_df = pd.read_csv("data/labeled_code/combined_data_uncommented.csv")


In [2]:
bq_df.head(10)

,Unnamed: 0.1,Unnamed: 0,line_count,comment_count,comment_total_len,comment_avg_len,comment_density,id_total,lower_case,id_total_var,...,filepath,forks,issue_events,stars,parse_error,func_async_count,file,labels,content,uncommented_content
0,0,0,116.0,2.0,44.0,22.000000,0.017241,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,data/00/wikihouse/urls.py,19,#!/usr/bin/env python\n# -*- coding: utf-8 -*-...,""""""" Provides ``mapping`` of url paths to reque..."
1,1,1,363.0,24.0,1244.0,51.833333,0.066116,381.0,214.0,333.0,...,NaN,NaN,NaN,NaN,NaN,NaN,data/0rpc/zerorpc-python/zerorpc/events.py,-1,# -*- coding: utf-8 -*-\n# Open Source Initiat...,import msgpack\nimport gevent.pool\nimport gev...
2,2,2,13.0,1.0,21.0,21.000000,0.076923,3.0,3.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,data/0xadada/dockdj/app/manage.py,19,"#!/usr/bin/env python\n""""""Django's command lin...","""""""Django's command line utility.""""""\nimport o..."
3,3,3,34.0,0.0,0.0,0.000000,0.000000,26.0,18.0,26.0,...,NaN,NaN,NaN,NaN,NaN,NaN,data/1stvamp/hippybot/setup.py,16,"""""""Installer for hippybot\n""""""\n\nimport os\nc...","""""""Installer for hippybot\n""""""\nimport os\ncwd..."
4,4,4,11.0,1.0,21.0,21.000000,0.090909,3.0,3.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,data/2buntu/2buntu-blog/manage.py,19,#!/usr/bin/env python\nimport os\nimport sys\n...,import os\nimport sys\nif __name__ == '__main_...
5,5,5,27.0,1.0,23.0,23.000000,0.037037,23.0,9.0,22.0,...,NaN,NaN,NaN,NaN,NaN,NaN,data/2buntu/2buntu-blog/twobuntu/categories/mi...,21,# -*- coding: utf-8 -*-\nfrom __future__ impor...,from __future__ import unicode_literals\nfrom ...
6,6,6,36.0,0.0,0.0,0.000000,0.000000,22.0,18.0,21.0,...,NaN,NaN,NaN,NaN,NaN,NaN,data/2buntu/2buntu-blog/twobuntu/news/views.py,2,import twitter\nfrom django.contrib import mes...,import twitter\nfrom django.contrib import mes...
7,7,7,35.0,15.0,734.0,48.933333,0.428571,3.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,data/2degrees/django-wsgi/django_wsgi/exc.py,-1,# -*- coding: utf-8 -*-\n#####################...,"""""""\nExceptions raised by :mod:`django_wsgi.`\..."
8,8,8,29.0,0.0,0.0,0.000000,0.000000,31.0,8.0,29.0,...,NaN,NaN,NaN,NaN,NaN,NaN,data/2gis/badger-api/common/storage.py,-1,import boto\nimport boto.s3.connection\n\nfrom...,import boto\nimport boto.s3.connection\nfrom d...
9,9,9,22.0,0.0,0.0,0.000000,0.000000,40.0,28.0,37.0,...,NaN,NaN,NaN,NaN,NaN,NaN,data/2gis/badger-api/stages/models.py,-1,from django.db import models\nimport datetime\...,from django.db import models\nimport datetime\...


In [ ]:
bq_df['uncomped_content'] = bq_df['content'].apply(for_loop)

In [33]:
bq_df.to_csv("data/combined_data_uncommented.csv") #remember to tell karl to dropna

In [ ]:
eval_comment_set_df = eval_set_df.copy()

processed_scripts = []
uncommented_scripts = []
for idx, script in enumerate(tqdm(eval_class_set_df['content'])):
    processed_scripts += [uncomment(script)]
    
eval_comment_set_df['uncommented_content'] = processed_scripts
eval_comment_set_df.to_csv("data/eval_parallel_corpora/eval_set_no_comment.csv")

## Classes

In [7]:
def is_init_func(func_node):
    return func_node.name == "__init__"
def is_super_expr(expr_node):
    for node in ast.walk(expr_node):
        if hasattr(node, "func") and type(node.func) == ast.Name:
            if node.func.id == "super":
                return True
    return False

In [8]:
import astunparse
import ast
def extract_func_from_class_node(class_node, no_super):
    func_list = class_node.body
    for func_node in func_list:
        if type(func_node) != ast.FunctionDef:
            continue
        
        if no_super and is_init_func(func_node):
            new_init_func_body = []
            for node in func_node.body:
                if is_super_expr(node): 
                    continue
                new_init_func_body += [node]
            func_node.body = new_init_func_body
    
        arg_list = func_node.args.args
        new_arg_list = []
        for arg in arg_list:
            if arg.arg == "self" or arg.arg == "cls":
                continue
            new_arg_list += [arg]
        func_node.args.args = new_arg_list
    return func_list
def remove_class_from_ast(ast_tree, no_super):
    class_nodes = []
    for node in ast.walk(ast_tree):
        for idx, child in enumerate(ast.iter_child_nodes(node)):
            if type(child) == ast.ClassDef:
                child.parent = node
                if type(node) in [ast.If, ast.Try, ast.For]:
                    if child in node.body:
                        # it is in the if
                        child.idx = node.body.index(child)
                    elif child in node.orelse:
                        # it is in the else
                        child.idx = node.orelse.index(child)
                        child.is_else = True
                    elif child in node.finalbody:
                        child.idx = node.finalbody.index(child)
                        child.is_final = True
                    else:
                        raise(f"Not in the body, another speciall case may happen, please look into this node: {ast.dump(node)}")
                else:
                    child.idx = node.body.index(child)
                    
                class_nodes = [child] + class_nodes
    
    if len(class_nodes) == 0:
        # nothing to change
        return None
    
    for class_node in class_nodes:
        func_list = extract_func_from_class_node(class_node, no_super)
        idx = class_node.idx
        
        # addressing classes in the else condition
        if hasattr(class_node, "is_else") and class_node.is_else:
            class_node.parent.orelse.pop(idx)
            class_node.parent.orelse = class_node.parent.orelse[:idx] + func_list + class_node.parent.orelse[idx:]
        elif hasattr(class_node, "is_final") and class_node.is_final:
            class_node.parent.finalbody.pop(idx)
            class_node.parent.finalbody = class_node.parent.finalbody[:idx] + func_list + class_node.parent.finalbody[idx:]
        else:
            class_node.parent.body.pop(idx)
            class_node.parent.body = class_node.parent.body[:idx] + func_list + class_node.parent.body[idx:]
            
    return ast_tree

def remove_self_cls_str(script):
    return script.replace("self.", "").replace("cls.", "")

def remove_class(script, no_super=False):
    ast_tree = ast.parse(script)
    processed_ast_tree = remove_class_from_ast(ast_tree, no_super)
    if processed_ast_tree:
        return remove_self_cls_str(astunparse.unparse(processed_ast_tree))
    return None

In [ ]:

# the removing classes method only works on python3 code, so we exclude py150k here
bq_no_outlier_df = pd.read_csv("data/labeled_code/bq_data_no_outlier.csv")
processed_scripts = []
for script in tqdm(bq_no_outlier_df['content']):
    try:
        processed_script = remove_class(script)
        processed_scripts += [processed_script]
    except SyntaxError:
        processed_scripts += [None]
    except Exception as e:
        print(script)
        print(e)
        raise(e)
bq_no_outlier_df['no_class_content'] = processed_scripts
bq_no_outlier_df.to_csv("data/labeled_code/bq_data_no_outlier_no_class.csv")

In [3]:

bq_outlier_df = pd.read_csv("data/labeled_code/bq_data_outlier.csv")

In [ ]:
processed_scripts = []
for idx, script in enumerate(tqdm(bq_outlier_df['content'])):
    if "Class" not in script:
        processed_scripts += [None]
        continue
    # if "__init__" not in script:
    #     continue
    try:
        processed_script = remove_class(script)
        processed_scripts += [processed_script
    except SyntaxError:
        processed_scripts += [None]
    except Exception as e:
        print(script)
        print(e)
        raise(e)
    # break
    
bq_outlier_df['no_class_content'] = processed_scripts
bq_outlier_df.to_csv("data/labeled_code/bq_data_outlier_no_class.csv")

In [ ]:
bq_outlier_df = pd.read_csv("data/labeled_code/bq_data_outlier.csv")

In [ ]:
processed_scripts = []
for idx, script in enumerate(tqdm(bq_outlier_df['content'])):
    if "Class" not in script:
        processed_scripts += [None]
        continue
    # if "__init__" not in script:
    #     continue
    try:
        processed_script = remove_class(script, no_super=True)
        processed_scripts += [processed_script]
    except SyntaxError:
        processed_scripts += [None]
    except Exception as e:
        print(script)
        print(e)
        raise(e)
    # break
    
bq_outlier_df['no_class_content'] = processed_scripts
bq_outlier_df.to_csv("data/labeled_code/bq_data_outlier_no_class_no_super.csv")

In [5]:
eval_set_df = pd.read_csv("data/evaluation_set.csv")

In [9]:
import math
eval_class_set_df = eval_set_df.copy()

processed_scripts = []
uncommented_scripts = []
for idx, script in enumerate(tqdm(eval_class_set_df['content'])):
    uncommented_scripts += [uncomment(script)]
    try:
        if type(script) != str:
            raise(SyntaxError)
        processed_script = remove_class(script)
        processed_scripts += [processed_script]
    except SyntaxError:
        processed_scripts += [None]
    except Exception as e:
        print(script)
        print(e)
        raise(e)
eval_class_set_df['no_class_content'] = processed_scripts
eval_class_set_df['content'] = uncommented_scripts
eval_class_set_df.to_csv("data/eval_parallel_corpora/eval_set_no_class.csv")

  0%|          | 0/393648 [00:00<?, ?it/s]

KeyboardInterrupt: 

## Docstring

In [12]:
import ast
test = "import ast\n class MyClass: \n\t\"\"\"A simple example class\"\"\" \n\ti = 12345 # le epic comment\n\tVAR = 1 \n\tWhoa = [i + 1 for i in range(0,10)]\n\n\tdef f(self):\n\t\treturn 'hello world'"
print (test)  #"\"\"\"start doc\"\"\"\n
#import astor
def undocstring(source):
    try:
        parsed = ast.parse(source)
        
        for node in ast.walk(parsed):
            print("Node is : ",node)
            #print("Node value is : ",node.body[0].value.s)
                    
            if not isinstance(node, (ast.Module)): #, ast.FunctionDef, ast.ClassDef, ast.AsyncFunctionDef
                continue

            if not len(node.body):
                continue

            if not isinstance(node.body[0], ast.Expr):
                continue

            if not hasattr(node.body[0], 'value') or not isinstance(node.body[0].value, ast.Str):
                continue

            # Uncomment lines below if you want print what and where we are removing
            # 
            # 

            node.body = node.body[1:]
        class toLower(ast.NodeTransformer):

            def visit_arg(self, node):
                return ast.arg(**{**node.__dict__, 'arg':node.arg.lower()})
            def visit_Name(self, node):
                #print("node id is : ",node.id)
                return ast.Name(**{**node.__dict__, 'id':node.id.lower()})

        new_code = ast.unparse(parsed)#toLower().visit(parsed))
        #print(new_code)
        return new_code
    except:
        parsed = 'nan'
        return parsed

import ast
 class MyClass: 
	"""A simple example class""" 
	i = 12345 # le epic comment
	VAR = 1 
	Whoa = [i + 1 for i in range(0,10)]

	def f(self):
		return 'hello world'


In [ ]:
bq_df = pd.read_csv(fname_prefix + "data/labeled_code/bq_data_outlier.csv")
bq_df['no_docstring_content'] = bq_df['content'].apply(undocstring)
bq_df['content'] = bq_df['content'].apply(uncomment)
bq_df.to_csv(fname_prefix + "data/labeled_code/bq_no_docstring_outlier.csv") 

In [13]:
eval_set_no_docstring_df = eval_set_df.copy()

processed_scripts = []
uncommented_scripts = []
for idx, script in enumerate(tqdm(eval_set_no_docstring_df['content'])):
    uncommented_scripts += [uncomment(script)]
    processed_scripts += [undocstring(script)]
    
eval_set_no_docstring_df['no_docstring_content'] = processed_scripts
eval_set_no_docstring_df['content'] = uncommented_scripts
eval_set_no_docstring_df.to_csv("data/eval_parallel_corpora/eval_set_no_docstring.csv")

  0%|          | 0/393648 [00:00<?, ?it/s]

## List Comps

In [11]:
test

'class MyClass: \n\t"""A simple example class""" \n\ti = 12345 # le epic comment\n\twhoa = [i + 1 for i in range(0,10)]\n\n\tdef f(self):\n\t\treturn \'hello world\''

In [15]:
import ast
#import astor

parsed = ast.parse(open('source.py').read())

for node in ast.walk(parsed):
    #print("Inside For")
    # let's work only on functions & classes definitions
    if not isinstance(node, (ast.ListComp)):
        continue
    for e in node.generators:
        print(e)

    if not len(node.generators):
        continue

    # Uncomment lines below if you want print what and where we are removing
    print(node)
    print(node.elt)
    print(node.generators[0].target.id)
    print(node.generators[0].target.id)



    # node.body = node.body[1:]

print('***** Processed source code output ******\n=========================================')

#print(astor.to_source(parsed))
    
class toForLoop(ast.NodeTransformer):
    def visit_arg(self, node):
        print(node.id)
        #node.arg.replace("_","")
        return ast.arg(**{**node.__dict__, 'arg':node.arg.lower()})
    def visit_ListComp(self, node):
        print(node.body)
        #return ast.Name(**{**node.__dict__, 'id':node.id.lower()})

toForLoop().visit_ListComp(ast.parse(parsed))
# new_code = ast.unparse(toLower().visit_Name(ast.parse(parsed)))
# print(new_code)

FileNotFoundError: [Errno 2] No such file or directory: 'source.py'

In [17]:
import ast

def for_loop(text):
    def wrap_if(body, compare):
        return  ast.If(
                    test= compare,
                    body=[body],
                    orelse=[])
    
    def wrap_for(body, a, index):
        return ast.For(
                    target = a.value.generators[index].target,
                    iter = a.value.generators[index].iter,
                    body = body,
                    lineno = a.lineno + 1,
                    orelse = [])
    
    def comp_to_expl(tree):
       if hasattr(tree, 'body'):
          i = 0
          while i < len(tree.body):
            if isinstance(a:=tree.body[i], ast.Assign) and isinstance(a.value, ast.ListComp):
                for_loop_body = [ast.Expr(
                                 value = ast.Call(
                                           func = ast.Attribute(value = ast.Name(id = a.targets[0].id), attr = 'append', ctx = ast.Load()),
                                           args = [a.value.elt],
                                           keywords = [] ))]
                
                for ind, for_loop in enumerate(a.value.generators[::-1]):
                    ind = len(a.value.generators) - 1 - ind
                    for if_state in a.value.generators[ind].ifs:
                        for_loop_body = wrap_if(for_loop_body, if_state)
                        
                    for_loop_body = wrap_for(for_loop_body, a, ind)                    

                
                tree.body = tree.body[:i] + \
                    [ast.Assign(
                       targets=[ast.Name(id = a.targets[0].id)], value = ast.List(elts = []),
                       lineno = a.lineno
                    )] + \
                    [for_loop_body] + \
                    tree.body[i+1:]   
                i += 1   
            i += 1
            
       for i in getattr(tree, '_fields', []):
          if isinstance(v:=getattr(tree, i, None), list):
             for i in v: 
                comp_to_expl(i)
          elif isinstance(v, ast.AST):
             comp_to_expl(v)

    try:
        parsed = ast.parse(text)
    except:
        return "Nan"
    
    try:
        comp_to_expl(parsed)
    except:
        pass
    
    return ast.unparse(parsed)
# print(for_loop(open('source.py').read()))

In [28]:
bq_df['uncomped_content'] = bq_df['uncommented_content'].apply(for_loop)

In [ ]:
bq_df.head(10)

In [37]:
sum(bq_df['uncommented_content'] != bq_df['uncomped_content'])

26784

In [18]:
eval_set_comp_df = eval_set_df.copy()

processed_scripts = []
uncommented_scripts = []
for idx, script in enumerate(tqdm(eval_set_df['content'])):
    uncommented_scripts += [uncomment(script)]
    processed_scripts += [for_loop(script)]

    break
    
eval_set_comp_df['no_docstring_content'] = processed_scripts
eval_set_comp_df['content'] = uncommented_scripts
eval_set_comp_df.to_csv("data/eval_parallel_corpora/eval_set_comp.csv")

  0%|          | 0/393648 [00:00<?, ?it/s]

ValueError: Length of values (1) does not match length of index (393648)

# Casing

In [3]:
#note get exact function anish used to generate the data this is old data
import ast
#import astor


print('***** Processed source code output ******\n=========================================')

#print(astor.to_source(parsed))



class toLower(ast.NodeTransformer):

    def visit_arg(self, node):

        return ast.arg(**{**node.__dict__, 'arg':node.arg.lower().replace("_","")})
    def visit_Name(self, node):
        return ast.Name(**{**node.__dict__, 'id':node.id.lower()})
    


class toFuncLower(ast.NodeTransformer):
    def visit_FunctionDef(self, node):
        return ast.FunctionDef(**{**node.__dict__, 'name':node.name.lower().replace("_","")})



***** Processed source code output ******


In [23]:
def uncasing(script):

    parsed = ast.parse(script)
    for node in ast.walk(parsed):
        if not isinstance(node, (ast.FunctionDef, ast.ClassDef, ast.AsyncFunctionDef)):
            continue

        if not len(node.body):
            continue

        if not isinstance(node.body[0], ast.Expr):
            continue

        if not hasattr(node.body[0], 'value') or not isinstance(node.body[0].value, ast.Str):
            continue

        # Uncomment lines below if you want print what and where we are removing
        # print("Node is : ",node)
        # print("Node value is : ",node.body[0].value.s)

        node.body = node.body[1:]
        
    new_code = astunparse.unparse(toLower().visit(parsed))
    #print(new_code)
    parsed_new = ast.parse(new_code)
    final_code = astunparse.unparse(toFuncLower().visit(parsed_new))
    return final_code

In [24]:
sample = eval_set_df["content"][5]

In [26]:
print(uncasing(sample))


'security converge saved queries\n\nRevision ID: e38177dbf641\nRevises: a8173232b786\nCreate Date: 2020-11-20 14:24:03.643031\n\n'
revision = 'e38177dbf641'
down_revision = 'a8173232b786'
from alembic import op
from sqlalchemy.exc import SQLAlchemyError
from sqlalchemy.orm import Session
from superset.migrations.shared.security_converge import add_pvms, get_reversed_new_pvms, get_reversed_pvm_map, migrate_roles, Pvm
new_pvms = {'SavedQuery': ('can_read', 'can_write')}
pvm_map = {pvm('SavedQueryView', 'can_list'): (pvm('SavedQuery', 'can_read'),), pvm('SavedQueryView', 'can_show'): (pvm('SavedQuery', 'can_read'),), pvm('SavedQueryView', 'can_add'): (pvm('SavedQuery', 'can_write'),), pvm('SavedQueryView', 'can_edit'): (pvm('SavedQuery', 'can_write'),), pvm('SavedQueryView', 'can_delete'): (pvm('SavedQuery', 'can_write'),), pvm('SavedQueryView', 'muldelete'): (pvm('SavedQuery', 'can_write'),), pvm('SavedQueryView', 'can_mulexport'): (pvm('SavedQuery', 'can_read'),), pvm('SavedQueryViewAp